In [2]:
import warnings
# import pandas as pd
# from bs4 import BeautifulSoup
# import nltk
warnings.filterwarnings('ignore')


In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import nltk


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('punkt')  # Download for sentence tokenization
nltk.download('stopwords')  # Download for stopword removal

In [4]:
df = pd.read_csv('Reviews.csv').head(1000)
# def clean_html(text):
#     if isinstance(text, str):
#         soup = BeautifulSoup(text, 'html.parser')
#         return soup.get_text()
#     return str(text)

# def tokenize_and_remove_stopwords(text):
#     tokens = nltk.word_tokenize(text)
#     filtered_tokens = [w for w in tokens if w.lower() not in stop_words]
#     return filtered_tokens

# # Define the enhanced preprocessing function
# def preprocess_text(text):
#     if isinstance(text, str):
#         text = text.lower()
#         tokens = tokenize_and_remove_stopwords(text)
#         tokens = [word for word in tokens if word not in stop_words]
#         tokens = [word.translate(punctuation_table) for word in tokens]
#         tokens = [word for word in tokens if word.strip()]
#         return ' '.join(tokens)
#     return str(text)

In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import nltk
import string

# Initialize NLTK resources before your functions
nltk.download('punkt')
nltk.download('stopwords')

# Get the list of English stopwords
stop_words = set(nltk.corpus.stopwords.words('english'))
punctuation_table = str.maketrans('', '', string.punctuation)

def clean_html(text):
    if isinstance(text, str):
        soup = BeautifulSoup(text, 'html.parser')
        return soup.get_text()
    return str(text)


# # Apply preprocessing to your DataFrame
# df['Processed_Summary'] = df['Summary'].apply(clean_html)
# df['Processed_Summary'] = df['Processed_Summary'].apply(preprocess_text)  # Enhanced preprocessing

# df['Processed_Text'] = df['Text'].apply(clean_html)
# df['Processed_Text'] = df['Processed_Text'].apply(preprocess_text)  # Enhanced preprocessing


def tokenize_and_remove_stopwords(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [w for w in tokens if w.lower() not in stop_words]
    return filtered_tokens

# Define the enhanced preprocessing function
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        tokens = tokenize_and_remove_stopwords(text)
        tokens = [word for word in tokens if word not in stop_words]
        tokens = [word.translate(punctuation_table) for word in tokens]
        tokens = [word for word in tokens if word.strip()]
        return ' '.join(tokens)
    return str(text)

# # Apply preprocessing to your DataFrame
# df['Processed_Summary'] = df['Summary'].apply(clean_html)
# df['Processed_Summary'] = df['Processed_Summary'].apply(preprocess_text)  # Enhanced preprocessing

# df['Processed_Text'] = df['Text'].apply(clean_html)
# df['Processed_Text'] = df['Processed_Text'].apply(preprocess_text)  # Enhanced preprocessing


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ecesoclab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = df.reset_index(drop=True)


In [7]:
from sklearn.model_selection import train_test_split

# Split the data
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)


In [8]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Now you can use the tokenizer with padding
inputs = tokenizer("Example text", return_tensors="pt", padding=True)


#  def __getitem__(self, idx):
#         text = self.texts[idx]
#         summary = self.summaries[idx]
#         encoding = tokenizer(
#             text, summary,
#             add_special_tokens=True,
#             max_length=self.max_length,
#             padding='max_length',
#             truncation=True,
#             return_tensors='pt'
#         )
#         inputs = {
#             'input_ids': encoding.input_ids.flatten(),
#             'attention_mask': encoding.attention_mask.flatten(),
#             'labels': encoding.input_ids.flatten()  # Using input_ids for labels for simplicity
#         }
#         return inputs

In [10]:
from torch.utils.data import Dataset

class SummaryDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_length=512):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)


# # Apply preprocessing to your DataFrame
# df['Processed_Summary'] = df['Summary'].apply(clean_html)
# df['Processed_Summary'] = df['Processed_Summary'].apply(preprocess_text)  # Enhanced preprocessing

# df['Processed_Text'] = df['Text'].apply(clean_html)
# df['Processed_Text'] = df['Processed_Text'].apply(preprocess_text)  # Enhanced preprocessing

    def __getitem__(self, idx):
        text = self.texts[idx]
        summary = self.summaries[idx]
        encoding = tokenizer(
            text, summary,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        inputs = {
            'input_ids': encoding.input_ids.flatten(),
            'attention_mask': encoding.attention_mask.flatten(),
            'labels': encoding.input_ids.flatten()  # Using input_ids for labels for simplicity
        }
        return inputs

# Initialize datasets
train_dataset = SummaryDataset(train_df['Processed_Text'], train_df['Processed_Summary'], tokenizer)
test_dataset = SummaryDataset(test_df['Processed_Text'], test_df['Processed_Summary'], tokenizer)


In [11]:
train_df

Id   ProductId          UserId  \
0     83  B003ZFRKGO  A2VOZX7YBT0D6D   
1    992  B006F2NYI2   AN85IOOEI9IYB   
2    790  B0018DQFPC  A3JO81AUCLO2U7   
3    895  B0041QIHC2  A3L9WUD9W8TAFT   
4    399  B001ELL6O8   ALSAOZ1V546VT   
..   ...         ...             ...   
745  107  B004K2IHUO  A1ZKFQLHFZAEH9   
746  271  B000LKZK7C  A2VUB6YYTF234O   
747  861  B000VKYKTG  A1O9O18AE5HIOL   
748  436  B000G6RYNE  A15USNEAJUXOSH   
749  103  B004K2IHUO   AVCA516CFZ9HF   

                                     ProfileName  HelpfulnessNumerator  \
0                      Johnnycakes "Johnnycakes"                    15   
1                                           Jdub                     2   
2                                     Sara_Aiden                     1   
3                                     Julie Sisk                     1   
4    A Research It Maven "Just the facts please"                     1   
..                                           ...                   ...   
745                 S. J. Monson "world citizen"                     2   
746                                   Anel Lopez                     0   
747                                  Sweet Tooth                     2   
748                                   L. Schrank                     0   
749                                    S. Fowler                     0   

     HelpfulnessDenominator  Score        Time  \
0                        15      5  1325635200   
1                         2      5  1323734400   
2                         2      2  1290038400   
3                         2      5  1284595200   
4                         1      5  1200182400   
..                      ...    ...         ...   
745                       8      3  1236384000   
746                       0      4  1302739200   
747                       2      3  1283472000   
748                       0      5  1326067200   
749                       0      4  1335139200   

                                               Summary  \
0    Forget Molecular Gastronomy - this stuff rocke...   
1                                    Best sauce around   
2                            Cute Item... but expired!   
3                                         Best Coffee!   
4    Arrowhead Mills whole grain buttermilk Pancake...   
..                                                 ...   
745                                      disappointing   
746                                       Good product   
747                             Chocolate was all melt   
748                                          Delicious   
749                                              TART!   

                                                  Text  \
0    I know the product title says Molecular Gastro...   
1    I was browsing around on Amazon looking for gi...   
2    The whole idea of a Spongebob Ginger Bread Pin...   
3    This is the best coffee we've ever had.  I run...   
4    HEY! These are GREAT Waffles and Pancakes! We ...   
..                                                 ...   
745  not what I was expecting in terms of the compa...   
746  I only use raw sugar, it did seem a little sma...   
747  The chocolate on the stick was all melted and ...   
748  I love these chips, I buy the 24 pack once a m...   
749  The crust on these tarts are perfect.  My husb...   

                                     Processed_Summary  \
0    forget molecular gastronomy stuff rockes coffe...   
1                                    best sauce around   
2                                    cute item expired   
3                                          best coffee   
4    arrowhead mills whole grain buttermilk pancake...   
..                                                 ...   
745                                      disappointing   
746                                       good product   
747                                     chocolate melt   
748                                          delicious   
749

In [12]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10
# )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Start training
trainer.train()



  2%|▏         | 10/564 [01:05<1:00:42,  6.58s/it]

{'loss': 7.2872, 'grad_norm': 217.26065063476562, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}


  4%|▎         | 20/564 [02:12<1:00:45,  6.70s/it]

{'loss': 6.6401, 'grad_norm': 239.51522827148438, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.11}


  5%|▌         | 30/564 [03:19<1:00:41,  6.82s/it]

...

 99%|█████████▉| 560/564 [1:07:54<00:29,  7.28s/it]

{'loss': 0.4764, 'grad_norm': 2.2932472229003906, 'learning_rate': 3.125e-06, 'epoch': 2.98}


100%|██████████| 564/564 [1:08:20<00:00,  7.27s/it]

{'train_runtime': 4100.4976, 'train_samples_per_second': 0.549, 'train_steps_per_second': 0.138, 'train_loss': 0.9462321941734205, 'epoch': 3.0}


TrainOutput(global_step=564, training_loss=0.9462321941734205, metrics={'train_runtime': 4100.4976, 'train_samples_per_second': 0.549, 'train_steps_per_second': 0.138, 'train_loss': 0.9462321941734205, 'epoch': 3.0})

In [13]:
!pip install rouge-score




[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\ecesoclab\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
from rouge_score import rouge_scorer

# Function to generate summaries
def generate_summary(model, tokenizer, text, max_length=100):
    try:
        if text is None:
            print("Input text is None. Skipping summary generation.")
            return None

        input_ids = tokenizer.encode(text, return_tensors='pt')
        if input_ids.size(1) > max_length:
            print(f"Skipping generation for input of length {input_ids.size(1)} as it exceeds max_length of {max_length}.")
            return None

        summary_ids = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
        return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# def compute_rouge_scores(model, dataset, tokenizer, dataframe):
#     scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
#     scores_list = []

#     # Loop through the dataset
#     for i in range(len(dataset)):
#         text = dataset.texts[i]
#         reference_summary = dataset.summaries[i]

#         # Generate summary
#         generated_summary = generate_summary(model, tokenizer, text)

#         # Compute ROUGE scores
#         scores = scorer.score(reference_summary, generated_summary)
        
#         # Create a detailed score dictionary
#         detailed_scores = {
#             'ROUGE-1 Precision': scores['rouge1'].precision,
#             'ROUGE-1 Recall': scores['rouge1'].recall,
#             'ROUGE-1 F1': scores['rouge1'].fmeasure,
#             'ROUGE-2 Precision': scores['rouge2'].precision,
#             'ROUGE-2 Recall': scores['rouge2'].recall,
#             'ROUGE-2 F1': scores['rouge2'].fmeasure,
#             'ROUGE-L Precision': scores['rougeL'].precision,
#             'ROUGE-L Recall': scores['rougeL'].recall,
#             'ROUGE-L F1': scores['rougeL'].fmeasure
#         }

#         # Append the score dictionary to the list
#         scores_list.append(detailed_scores)

#     # Append scores to the dataframe
#     score_df = pd.DataFrame(scores_list)
#     enhanced_df = pd.concat([dataframe.reset_index(drop=True), score_df], axis=1)
#     return enhanced_df

# # Call the function and pass the DataFrame
# enhanced_df = compute_rouge_scores(model, test_dataset, tokenizer, test_df)

# Function to compute ROUGE scores
# Function to compute ROUGE scores
def compute_rouge_scores(model, dataset, tokenizer):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    # Loop through the dataset
    for i in range(len(dataset)):
        text = dataset.texts[i]
        reference_summary = dataset.summaries[i]
        # Generate summary
        generated_summary = generate_summary(model, tokenizer, text)

        # Check if generated summary is None
        if generated_summary is None:
            print(f"Skipping scoring for input {i} as generated summary is None.")
            continue

        # Compute ROUGE scores
        score = scorer.score(reference_summary, generated_summary)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)

    # Calculate average scores
    avg_scores = {
        'rouge1': sum(scores['rouge1']) / len(scores['rouge1']),
        'rouge2': sum(scores['rouge2']) / len(scores['rouge2']),
        'rougeL': sum(scores['rougeL']) / len(scores['rougeL'])
    }

    return avg_scores


# Example of how to call the compute_rouge_scores function
# This assumes you have a trained model and tokenizer ready, and a dataset prepared
avg_rouge_scores = compute_rouge_scores(model, test_dataset, tokenizer)
print("Average ROUGE Scores:", avg_rouge_scores)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Skipping generation for input of length 176 as it exceeds max_length of 100.
Skipping scoring for input 8 as generated summary is None.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Skipping generation for input of length 120 as it exceeds max_length of 100.
Skipping scoring for input 13 as generated summary is None.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Skipping generation for input of length 109 as it exceeds max_length of 100.
Skipping scoring for input 218 as generated summary is None.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Skipping generation for input of length 114 as it exceeds max_length of 100.
Skipping scoring for input 246 as generated summary is None.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Average ROUGE Scores: {'rouge1': 0.08501119642711284, 'rouge2': 0.02142304771114964, 'rougeL': 0.07844308079921475}


In [20]:
from rouge_score import rouge_scorer

def compute_rouge_scores(model, dataset, tokenizer, dataframe):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores_list = []

    # Loop through the dataset
    for i in range(len(dataset)):
        text = dataset.texts[i]
        reference_summary = dataset.summaries[i]

        # Generate summary
        generated_summary = generate_summary(model, tokenizer, text)

        if generated_summary is None:
            print(f"Skipping scoring for input {i} as generated summary is None.")
            continue

        # Compute ROUGE scores
        scores = scorer.score(reference_summary, generated_summary)


        # if generated_summary is None:
        #     print(f"Skipping scoring for input {i} as generated summary is None.")
        #     continue

        # # Compute ROUGE scores
        # scores = scorer.score(reference_summary, generated_summary)
        # Check if ROUGE scores are computed successfully
        if scores['rouge1'].precision is None or scores['rouge1'].recall is None or scores['rouge1'].fmeasure is None:
            print(f"Skipping scoring for input {i} as ROUGE scores could not be computed.")
            continue

        # Create a detailed score dictionary
        detailed_scores = {
            'ROUGE-1 Precision': scores['rouge1'].precision,
            'ROUGE-1 Recall': scores['rouge1'].recall,
            'ROUGE-1 F1': scores['rouge1'].fmeasure,
            'ROUGE-2 Precision': scores['rouge2'].precision,
            'ROUGE-2 Recall': scores['rouge2'].recall,
            'ROUGE-2 F1': scores['rouge2'].fmeasure,
            'ROUGE-L Precision': scores['rougeL'].precision,
            'ROUGE-L Recall': scores['rougeL'].recall,
            'ROUGE-L F1': scores['rougeL'].fmeasure
        }

        # Append the score dictionary to the list
        scores_list.append(detailed_scores)

    # Append scores to the dataframe
    score_df = pd.DataFrame(scores_list)
    final_df = pd.concat([dataframe.reset_index(drop=True), score_df], axis=1)
    return final_df

# Call the function and pass the DataFrame
final_df = compute_rouge_scores(model, test_dataset, tokenizer, test_df)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Skipping generation for input of length 176 as it exceeds max_length of 100.
Skipping scoring for input 8 as generated summary is None.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Skipping generation for input of length 120 as it exceeds max_length of 100.
Skipping scoring for input 13 as generated summary is None.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Skipping generation for input of length 109 as it exceeds max_length of 100.
Skipping scoring for input 218 as generated summary is None.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Skipping generation for input of length 114 as it exceeds max_length of 100.
Skipping scoring for input 246 as generated summary is None.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [21]:
# Save to CSV
final_df.to_csv("rouge_scores_report.csv", index=False)

final_df

Id   ProductId          UserId                   ProfileName  \
0    522  B000G6RYNE   AYBYYDVV5ABJE                      retrodog   
1    738  B001HTKZ5S  A3KJ6JZ8MWERPG                       RnnrGrl   
2    741  B001HTKZ5S  A2USOYPY0M1R6X                       Colleen   
3    661  B002BCD2OG  A3NM3RXJ8HI5TC                      C. Gibbs   
4    412  B00061KYVI  A28RMYGO9RGA0K                       marilyn   
..   ...         ...             ...                           ...   
245  110  B001REEG6C   AY12DBB0U420B                 Gary Peterson   
246  431  B000G6RYNE  A1S8ELUE383P8E  John F. Peters "John Peters"   
247   78  B004X2KR36  A26M5O53PHZTKN                 Debs "peanut"   
248   85  B0019CW0HE   AK2CXHH9VRZ2A                      I. GLENN   
249  287  B000H13270   AVQIRN6E7J7UA                       imdebra   

     HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                       0                       1      3  1251244800   
1                       1                       1      1  1264896000   
2                       0                       0      1  1282608000   
3                       0                       0      5  1344297600   
4                       0                       0      5  1328140800   
..                    ...                     ...    ...         ...   
245                     0                       0      5  1316390400   
246                     0                       0      5  1330905600   
247                     0                       0      5  1348185600   
248                     4                       4      3  1313193600   
249                     0                       4      3  1280534400   

                                  Summary  \
0                         Too Much Flavor   
1                      Big Disappointment   
2    Cherrybrook Kitchen Vanilla Frosting   
3                            Great stuff!   
4                   french's roast'n bags   
..                                    ...   
245          My Idea of a Good Diet Food.   
246                     Ridiculously Good   
247                           Taste great   
248             INCREASED MY DOGS ITCHING   
249            Not mild enough for me lol   

                                                  Text  ...  \
0    These things are just too darn cheesy. If you ...  ...   
1    I love Cherrybrook Kitchen and have tried almo...  ...   
2    I recently purchased this on sale at my local ...  ...   
3    I love using this to sweeten tea, coffee, etc....  ...   
4    i love this product cant find it locally. Some...  ...   
..                                                 ...  ...   
245  I'm presently on a diet and I was at my Fresh ...  ...   
246  I cannot say how much I love Kettle brand pota...  ...   
247  These taste really good. I have been purchasin...  ...   
248  Awesome dog food. However, when given to my "B...  ...   
249  Nothing wrong with the product, but I sure had...  ...   

                                        Processed_Text ROUGE-1 Precision  \
0    things darn cheesy like lot flavor ll love oth...          0.083333   
1    love cherrybrook kitchen tried almost products...          0.000000   
2    recently purchased sale local grocery store kn...          0.013699   
3    love using sweeten tea coffee etc tastes much ...          0.000000   
4    love product cant find locally sometimes littl...          0.000000   
..                                                 ...               ...   
245  m presently diet fresh easy neighborhood groce...               NaN   
246  say much love kettle brand potato chips kettle...               NaN   
247  taste really good purchasing different brand s...               NaN   
248  awesome dog food however given boston severe r...               NaN   
249  nothing wrong product sure mix lots ingredient...               NaN   

     ROUGE-1 Recall  ROUGE-1 F1  ROUGE-2 Precision  ROUGE-2 Recall  \
0              0.50